### Talk Time Data Prepare

In [1]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from nltk import ngrams
from functools import reduce
from pathlib import Path
import shutil

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [3]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "Intersect"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0 native word start index
word_end = 200  # 28 native word end index

# word usage
min_word_use_num = 1
max_word_use_num = 6

# youtube
sample_num = 6
time_shift = 0.6

In [4]:
def word_in_wordgroup_simple(source_word_list, df_target, target_column, word_sample_num, simple=False):

    '''word_in_wordgroup(not_in_sent_word_list, df_youtube_sent_select, "search_string", 5, simple=False)\n
       source_word_list is searching word list\n
       df_target is dataframe, target_column are dataframe column string name\n
       word_sample_num is searching sample number.
       simple use for all column row result or only target column result 
    '''
    if simple:
        df_select = df_target[[f"{target_column}"]].dropna()
    else:
        df_select = df_target
        
    df_result = pd.DataFrame()
    for i in source_word_list:
        try:
            word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].head(word_sample_num)    
        except:
            pass        
        word_in_word_cluster.insert(0,"search_string",i)
        df_result = pd.concat([df_result,word_in_word_cluster], axis=0)
    df_result.reset_index(drop=True, inplace=True)

    return df_result

In [5]:
def word_group_youtube(df, search_list, target_column, sample_num):
    '''
    word_group_youtube(df_youtube_sentence, search_list, "sentence", 6)\n
    df_youtube_sentence is dataframe and "sentence" is its column for external searching_list
    ''' 
    df_search_result = pd.DataFrame()
    for j in search_list:
        try:
            df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)].sample(sample_num)
        except:
            df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)].head(sample_num)
        #df_result = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)]  # sentence length part
        #df_result.sort_values(f"{target_column}",key=lambda x:x.str.len(), inplace=True)
        #df_select = df_result.head(sample_num)
        df_select.insert(0,"search_string",j)
        df_search_result = pd.concat([df_search_result,df_select], axis=0)
    df_search_result.reset_index(inplace=True, drop=True)
    
    return df_search_result

In [6]:
def word_group_time_loc(df, search, start_sent, end_sent, sent, sent_video_id):
    '''
    word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")\n
    df_search_result is dataframe and "search_string", "start_time", "end_time", "sentence", "video_id" are its columns
    '''
    df.reset_index(drop=True, inplace=True)
    word_time_loc_list = []
    for i in range(len(df)):
        word = df.loc[i,f"{search}"]
        start_time = df.loc[i,f"{start_sent}"]
        end_time = df.loc[i,f"{end_sent}"]
        sentence = df.loc[i,f"{sent}"]
        video_id = df.loc[i,f"{sent_video_id}"]
        time_length = end_time-start_time
        sentence_length = len(sentence)
        time_length_ratio = time_length/sentence_length
        loc_list = []
        for j in re.finditer(fr"(?:\s|^){word}(?:\s|$)", sentence, re.IGNORECASE|re.UNICODE):
            loc_list.append(j)
            start = loc_list[0].start()
            end = loc_list[0].end()
            start_loc = start_time+(start*time_length_ratio)
            end_loc = start_time+(end*time_length_ratio)
        word_time_loc_list.append([word,start_loc,end_loc,sentence,video_id])
    df_word_time_loc = pd.DataFrame(word_time_loc_list, columns=[f"{search}",f"{start_sent}",f"{end_sent}",f"{sent}",f"{sent_video_id}"])

    return df_word_time_loc

In [7]:
def word_count_result(df, column_list, set_condition=False): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_bool(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        if set_condition:
            var_list = set(df[f"{i}"].dropna().tolist())
        else:
            var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    df_word_count.sort_values("word_count", ascending=False, inplace=True)
    df_word_count.reset_index(inplace=True, drop=True)
    
    return  df_word_count

In [8]:
def word_usage_result(word_list, df_target, target_column, target_opt_column, word_usage_min, word_usage_max):
    '''
    word_usage_result(word_list, df_threegram_pair, "threegram", "frequency", 1, 5) \n
    word_list is a list, df_target is a dateframe, target_column is df_target dataframe target column, \n
    target_opt_column is df_target dataframe opt_target column, \n
    word_usage_min and word_usage_max word usage condition.
    '''    
    word_num_dict = {}
    for i in word_list:
        word_num_dict[f"{i}"] = 0
    
    result_list_select = []
    var_list = []
    for i in range(len(df_target)):
        target_value = df_target.loc[i,f"{target_column}"]
        opt_value = df_target.loc[i,f"{target_opt_column}"]
        words = word_tokenize(target_value)   
        temp_list = [word for word in words]
        temp_list = temp_list + var_list
        # word count for max
        dict_list_count = Counter(temp_list)
        count_list = list(dict_list_count.values())
        # word count for min
        count_list2 = list(word_num_dict.values())
    
        if any([True if i>word_usage_max else False for i in count_list]) or not(any([True if j<word_usage_min else False for j in count_list2])):
            pass
        else:
            var_list = temp_list
            result_list_select.append([target_value,opt_value]) 
    
            for item2 in dict_list_count.items(): 
                word_num_dict[item2[0]] = item2[1]        
    df_result = pd.DataFrame(result_list_select, columns=[f"{target_column}",f"{target_opt_column}"])
    df_result.sort_values(by="frequency", ascending=False, inplace=True)
    df_result.reset_index(drop=True, inplace=True)
    
    return df_result

In [9]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Talk Time/Data/1-Talk Time Data Prepare/{lang_folder.capitalize()}"

Path(path).mkdir(parents=True, exist_ok=True)

In [10]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
987924,karneleme,5
987925,karnaya,5
987926,dörtlümüzün,5
987927,karnavalınız,5


In [11]:
df_word_select = df_word_all.iloc[word_start:word_end,]

In [12]:
df_word_select

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
195,saat,399989
196,onunla,399330
197,yapıyorsun,398274
198,neler,397377


In [13]:
df_word_select.to_excel(f"{lang_folder.capitalize()}_200_Word.xlsx", index=False)

In [14]:
word_list = df_word_select["word"].to_list()
len(word_list)

200

In [15]:
df_twogram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/N Gram And Sentence/Two_Gram_And_Sentence.csv")  # ext. sentence and ngram
df_twogram_sent.rename(columns={"two_gram":"twogram"}, inplace=True)  # ext. Not: Two_Gram_And_Sentence.csv convert to Two_Gram_And_Sentence_All.csv
df_twogram_sent

,twogram,frequency
0,teşekkür ederim,244149
1,öyle mi,209900
2,ne oldu,195799
3,aman tanrım,189521
4,özür dilerim,153784
...,...,...
1036515,güzeldi tommy,3
1036516,durumu tuhaflaştırma,3
1036517,güzeldi canım,3
1036518,güzeldi daniel,3


In [16]:
# Twogram Func
d_list2  = df_twogram_sent.iloc[:,0].values.tolist()

resultlist2 = []
manager = multiprocessing.Manager()
resultlist2 = manager.list()

def word_in_wordgroup(d_list2):
    mergelist2 = []
    try:
        word = d_list2.split()
    except:
        word = []
        #pass  disabled for non split value
    var1 = range(len(word))
    for j in var1:
        if word[j] in word_list:
            mergelist2.append(word[j])
            if len(mergelist2) == len(word):
                    resultlist2.append(d_list2)
                        
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, d_list2) # string_word liste

result_list2 = list(resultlist2)
df_result2 = pd.DataFrame(result_list2, columns=[0])  # add columns parameter for empty result
df_result2 = df_result2.rename(columns = {0: "twogram"})
df_merge2 = pd.merge(df_result2, df_twogram_sent, how="left", on="twogram")
df_merge_result2 = df_merge2.sort_values(by="frequency", ascending=False)
df_merge_result2.drop_duplicates(inplace=True)
df_merge_result2.reset_index(drop=True, inplace=True)
df_twogram_select = df_merge_result2
df_twogram_select

,twogram,frequency
0,teşekkür ederim,244149
1,öyle mi,209900
2,ne oldu,195799
3,ne yapıyorsun,140797
4,gerçekten mi,137106
...,...,...
8463,belki tabii,3
8464,aslında kötü,3
8465,da olabilir,3
8466,değil mu,3


In [17]:
df_threegram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/N Gram And Sentence/Three_Gram_And_Sentence.csv")  # ext. sentence and ngram
df_threegram_sent.rename(columns={"three_gram":"threegram"}, inplace=True)  # ext. Not: Two_Gram_And_Sentence.csv convert to Two_Gram_And_Sentence_All.csv
df_threegram_sent

,threegram,frequency
0,ne demek istiyorsun,55905
1,bu da ne,50098
2,hayır hayır hayır,41386
3,sen iyi misin,32965
4,çok teşekkür ederim,27663
...,...,...
1039403,kapıyı kapat aaron,5
1039404,kardeşime bir hediye,5
1039405,sonra da rachampsı,5
1039406,usta lee neredesin,5


In [18]:
# Threegram Func
d_list3  = df_threegram_sent.iloc[:,0].values.tolist()

resultlist3 = []
manager = multiprocessing.Manager()
resultlist3 = manager.list()

def word_in_wordgroup(d_list3):
    mergelist3 = []
    try:
        word = d_list3.split()
    except:
        word = []
        #pass  disabled for non split value
    var1 = range(len(word))
    for j in var1:
        if word[j] in word_list:
            mergelist3.append(word[j])
            if len(mergelist3) == len(word):
                    resultlist3.append(d_list3)
                        
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, d_list3) # string_word liste

result_list3 = list(resultlist3)
df_result3 = pd.DataFrame(result_list3, columns=[0])  # add columns parameter for empty result
df_result3 = df_result3.rename(columns = {0: "threegram"})
df_merge3 = pd.merge(df_result3, df_threegram_sent, how="left", on="threegram")
df_merge_result3 = df_merge3.sort_values(by="frequency", ascending=False)
df_merge_result3.drop_duplicates(inplace=True)
df_merge_result3.reset_index(drop=True, inplace=True)
df_threegram_select = df_merge_result3
df_threegram_select

,threegram,frequency
0,ne demek istiyorsun,55905
1,bu da ne,50098
2,hayır hayır hayır,41386
3,sen iyi misin,32965
4,çok teşekkür ederim,27663
...,...,...
19277,bize şimdi söyle,5
19278,ne hakkında bilmiyorum,5
19279,anne tamam iyi,5
19280,öyle değil sanırım,5


In [19]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{lang_folder.capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,00:00:00.294,00:00:03.294,bu dizinin betimlemesi staff fi lm tarafından,fI1BI4d5KFU
1,00:00:03.375,00:00:06.375,sesli betimleme derneğine yaptırılmıştır,fI1BI4d5KFU
2,00:00:06.462,00:00:09.462,wwwsebederorg,fI1BI4d5KFU
3,00:00:10.267,00:00:11.394,mumu,fI1BI4d5KFU
4,00:00:11.835,00:00:14.061,bak şu an sinir katsayım hat safhada,fI1BI4d5KFU
...,...,...,...,...
3934202,00:10:00.240,00:10:07.818,videoyu bitirmeden şunu da söylemek isterim ki...,dRssR_apbR0
3934203,00:10:07.994,00:10:13.534,bu nedenle ulaşımda insanların yarısına yakını...,dRssR_apbR0
3934204,00:10:14.745,00:10:17.924,danimarkayla alakalı şimdilik söyleyeceklerim ...,dRssR_apbR0
3934205,00:10:18.047,00:10:22.398,video hoşunuza gittiyse beğenmeyi ve kanalıma ...,dRssR_apbR0


In [20]:
df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second 
df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])

In [21]:
df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())
df_youtube_sentence

In [ ]:
df_sentence_link = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Talk Time/Data/{lang_folder.capitalize()}/{lang_folder.capitalize()}_200_Word_Group_In_Youtube_Sentence_Word_Index_List_Sample_Selected.xlsx")
df_sentence_link = df_sentence_link.iloc[:,[0,1,2,3,4,5]]
df_sentence_link

,search_string,start_time,end_time,sentence,video_id,video_url
0,kendi şimdi sen mi ne oluyor biliyor musun bu ...,446,451,artık hamill olması lazım altı kendi şimdi sen...,Yuu5VhJBIUg,https://www.youtube.com/watch?v=Yuu5VhJBIUg&t=...
1,gerçekten uzun ve bu da benim için çok önemli ...,623,627,bu tabletin şarj ömrü batarya ömrü gerçekten u...,r_9ngjGh2Ks,https://www.youtube.com/watch?v=r_9ngjGh2Ks&t=...
2,her zaman öyle değil herkes için de her zaman ...,559,562,her zaman öyle değil herkes için de her zaman ...,cPoQ27hCNOU,https://www.youtube.com/watch?v=cPoQ27hCNOU&t=...
3,bile yok bu da benim için aslında önemli bir ş...,131,134,bulabilirsiniz şu an görünce bile yok bu da be...,aqQ7IUho9pA,https://www.youtube.com/watch?v=aqQ7IUho9pA&t=...
4,için artık değil misin gerek yok bunun için si...,65,70,göndermek için artık değil misin gerek yok bun...,gX77n13Qj0o,https://www.youtube.com/watch?v=gX77n13Qj0o&t=65s
...,...,...,...,...,...,...
64,gerçek mi ya ne,1471,1473,gerçek mi ya ne,kulZ_sWzBR0,https://www.youtube.com/watch?v=kulZ_sWzBR0&t=...
65,bence olur bence olmaz,3893,3895,bence olur bence olmaz,MhHCQ_cb3T8,https://www.youtube.com/watch?v=MhHCQ_cb3T8&t=...
66,ben asla öyle bir şey,4768,4769,ben asla öyle bir şey yapmadım,kulZ_sWzBR0,https://www.youtube.com/watch?v=kulZ_sWzBR0&t=...
67,bir şey yok ki ya adam,2554,2556,bir şey yok ki ya adam hırsızın teki çıktı işte,MhHCQ_cb3T8,https://www.youtube.com/watch?v=MhHCQ_cb3T8&t=...


In [ ]:
twogram_set = set()
threegram_set = set()
for sent in df_sentence_link["search_string"]:
    twogram_zip = ngrams(sent.split(), 2)
    for two in twogram_zip:
        twogram = " ".join(two)
        twogram_set.add(f"{twogram}") 
    threegram_zip = ngrams(sent.split(), 3)
    for three in threegram_zip:
        threegram = " ".join(three)
        threegram_set.add(f"{threegram}")

In [ ]:
len(twogram_set)

334

In [ ]:
len(threegram_set)

288

In [ ]:
twogram_list = list(twogram_set)
#twogram_list

In [ ]:
threegram_list = list(threegram_set)
#threegram_list

In [ ]:
df_twogram_for_link = df_twogram_select[df_twogram_select["twogram"].isin(twogram_list)]
df_twogram_for_link.reset_index(drop=True, inplace=True)
df_twogram_for_link

,twogram,frequency
0,teşekkür ederim,244149
1,neler oluyor,100618
2,evet efendim,86185
3,sorun değil,79391
4,çok güzel,61809
...,...,...
182,daha doğru,4
183,seni bir,4
184,tamam çok,4
185,bir yardım,4


In [ ]:
df_twogram_selected_for_link = word_usage_result(word_list, df_twogram_for_link, "twogram", "frequency", min_word_use_num, max_word_use_num)
df_twogram_selected_for_link 

,twogram,frequency
0,teşekkür ederim,244149
1,neler oluyor,100618
2,evet efendim,86185
3,sorun değil,79391
4,çok güzel,61809
...,...,...
157,sonra iki,5
158,bak hala,4
159,olur bence,4
160,daha doğru,4


In [ ]:
df_threegram_for_link = df_threegram_select[df_threegram_select["threegram"].isin(threegram_list)]
df_threegram_for_link.reset_index(drop=True, inplace=True)
df_threegram_for_link

,threegram,frequency
0,bir şey yok,14464
1,ben teşekkür ederim,2558
2,orada neler oluyor,2456
3,gerçekten çok güzel,1427
4,öyle bir şey,1332
5,ne oluyor ya,1180
6,tamam devam et,1142
7,ve bu da,912
8,ne kadar büyük,809
9,güzel çok güzel,704


In [ ]:
df_threegram_selected_for_link = word_usage_result(word_list, df_threegram_for_link, "threegram", "frequency", min_word_use_num, max_word_use_num)
df_threegram_selected_for_link 

,threegram,frequency
0,bir şey yok,14464
1,ben teşekkür ederim,2558
2,orada neler oluyor,2456
3,gerçekten çok güzel,1427
4,öyle bir şey,1332
5,ne oluyor ya,1180
6,tamam devam et,1142
7,ve bu da,912
8,ne kadar büyük,809
9,güzel çok güzel,704


In [ ]:
df_twogram_search_result = word_group_youtube(df_youtube_sentence, df_twogram_selected_for_link["twogram"], "sentence", sample_num)
df_twogram_search_result

,search_string,start_time,end_time,sentence,video_id
0,teşekkür ederim,2125.010,2127.723,çok zevk aldım gerçekten çok zevk aldım çok te...,ry0hEicwuhQ
1,teşekkür ederim,1492.400,1494.780,teşekkür ederim anneciğim iyiyim,8OjNg0Sspro
2,teşekkür ederim,427.690,433.960,musun peki çok teşekkür ederim hadi kendinize ...,cUfp2lydmho
3,teşekkür ederim,50.240,51.120,çok teşekkür ederim i lay,acHbcJ67OtM
4,teşekkür ederim,6817.522,6822.322,feride hanım her şey için çok teşekkür ederim ...,EBvJ2rWj7LM
...,...,...,...,...,...
952,senin gerçekten,2575.600,2583.650,bunu zaman zaman söylüyorum ama burada bahsett...,GimA7_uhyWg
953,senin gerçekten,4999.083,5000.251,mert senin gerçekten,g81mNCrBnO4
954,senin gerçekten,280.946,284.525,senin gerçekten klinik bir araştırmaya ihtiyac...,kOwaqR6St2I
955,senin gerçekten,975.300,981.800,senin gerçekten özel mistik gizemli bir insan ...,oKfaa43c0hA


In [ ]:
df_word_group_time_loc_twogram_result = word_group_time_loc(df_twogram_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_twogram_result

,search_string,start_time,end_time,sentence,video_id
0,teşekkür ederim,2126.987271,2127.723000,çok zevk aldım gerçekten çok zevk aldım çok te...,ry0hEicwuhQ
1,teşekkür ederim,1492.400000,1493.590000,teşekkür ederim anneciğim iyiyim,8OjNg0Sspro
2,teşekkür ederim,428.623830,429.757766,musun peki çok teşekkür ederim hadi kendinize ...,cUfp2lydmho
3,teşekkür ederim,50.345600,50.944000,çok teşekkür ederim i lay,acHbcJ67OtM
4,teşekkür ederim,6819.922000,6821.328897,feride hanım her şey için çok teşekkür ederim ...,EBvJ2rWj7LM
...,...,...,...,...,...
952,senin gerçekten,2580.085878,2581.130534,bunu zaman zaman söylüyorum ama burada bahsett...,GimA7_uhyWg
953,senin gerçekten,4999.316600,5000.251000,mert senin gerçekten,g81mNCrBnO4
954,senin gerçekten,280.946000,282.047231,senin gerçekten klinik bir araştırmaya ihtiyac...,kOwaqR6St2I
955,senin gerçekten,975.300000,976.442857,senin gerçekten özel mistik gizemli bir insan ...,oKfaa43c0hA


In [ ]:
df_word_group_time_loc_twogram_result.start_time = df_word_group_time_loc_twogram_result.start_time.apply(lambda x: (x-time_shift))
df_word_group_time_loc_twogram_result.end_time = df_word_group_time_loc_twogram_result.end_time.apply(lambda x: (x+time_shift))
df_word_group_time_loc_twogram_result

,search_string,start_time,end_time,sentence,video_id
0,teşekkür ederim,2126.387271,2128.323000,çok zevk aldım gerçekten çok zevk aldım çok te...,ry0hEicwuhQ
1,teşekkür ederim,1491.800000,1494.190000,teşekkür ederim anneciğim iyiyim,8OjNg0Sspro
2,teşekkür ederim,428.023830,430.357766,musun peki çok teşekkür ederim hadi kendinize ...,cUfp2lydmho
3,teşekkür ederim,49.745600,51.544000,çok teşekkür ederim i lay,acHbcJ67OtM
4,teşekkür ederim,6819.322000,6821.928897,feride hanım her şey için çok teşekkür ederim ...,EBvJ2rWj7LM
...,...,...,...,...,...
952,senin gerçekten,2579.485878,2581.730534,bunu zaman zaman söylüyorum ama burada bahsett...,GimA7_uhyWg
953,senin gerçekten,4998.716600,5000.851000,mert senin gerçekten,g81mNCrBnO4
954,senin gerçekten,280.346000,282.647231,senin gerçekten klinik bir araştırmaya ihtiyac...,kOwaqR6St2I
955,senin gerçekten,974.700000,977.042857,senin gerçekten özel mistik gizemli bir insan ...,oKfaa43c0hA


In [ ]:
df_word_group_time_loc_twogram_result.start_time = df_word_group_time_loc_twogram_result.start_time.apply(lambda x: round(x))
df_word_group_time_loc_twogram_result.end_time = df_word_group_time_loc_twogram_result.end_time.apply(lambda x: round(x))
df_word_group_time_loc_twogram_result 

,search_string,start_time,end_time,sentence,video_id
0,teşekkür ederim,2126,2128,çok zevk aldım gerçekten çok zevk aldım çok te...,ry0hEicwuhQ
1,teşekkür ederim,1492,1494,teşekkür ederim anneciğim iyiyim,8OjNg0Sspro
2,teşekkür ederim,428,430,musun peki çok teşekkür ederim hadi kendinize ...,cUfp2lydmho
3,teşekkür ederim,50,52,çok teşekkür ederim i lay,acHbcJ67OtM
4,teşekkür ederim,6819,6822,feride hanım her şey için çok teşekkür ederim ...,EBvJ2rWj7LM
...,...,...,...,...,...
952,senin gerçekten,2579,2582,bunu zaman zaman söylüyorum ama burada bahsett...,GimA7_uhyWg
953,senin gerçekten,4999,5001,mert senin gerçekten,g81mNCrBnO4
954,senin gerçekten,280,283,senin gerçekten klinik bir araştırmaya ihtiyac...,kOwaqR6St2I
955,senin gerçekten,975,977,senin gerçekten özel mistik gizemli bir insan ...,oKfaa43c0hA


In [ ]:
df_word_group_time_loc_twogram_result["video_url"] = "https://www.youtube.com/watch?v="+df_word_group_time_loc_twogram_result['video_id'].map(str)+"&t="+df_word_group_time_loc_twogram_result['start_time'].map(str)+"s"
df_word_group_time_loc_twogram_result

,search_string,start_time,end_time,sentence,video_id,video_url
0,teşekkür ederim,2126,2128,çok zevk aldım gerçekten çok zevk aldım çok te...,ry0hEicwuhQ,https://www.youtube.com/watch?v=ry0hEicwuhQ&t=...
1,teşekkür ederim,1492,1494,teşekkür ederim anneciğim iyiyim,8OjNg0Sspro,https://www.youtube.com/watch?v=8OjNg0Sspro&t=...
2,teşekkür ederim,428,430,musun peki çok teşekkür ederim hadi kendinize ...,cUfp2lydmho,https://www.youtube.com/watch?v=cUfp2lydmho&t=...
3,teşekkür ederim,50,52,çok teşekkür ederim i lay,acHbcJ67OtM,https://www.youtube.com/watch?v=acHbcJ67OtM&t=50s
4,teşekkür ederim,6819,6822,feride hanım her şey için çok teşekkür ederim ...,EBvJ2rWj7LM,https://www.youtube.com/watch?v=EBvJ2rWj7LM&t=...
...,...,...,...,...,...,...
952,senin gerçekten,2579,2582,bunu zaman zaman söylüyorum ama burada bahsett...,GimA7_uhyWg,https://www.youtube.com/watch?v=GimA7_uhyWg&t=...
953,senin gerçekten,4999,5001,mert senin gerçekten,g81mNCrBnO4,https://www.youtube.com/watch?v=g81mNCrBnO4&t=...
954,senin gerçekten,280,283,senin gerçekten klinik bir araştırmaya ihtiyac...,kOwaqR6St2I,https://www.youtube.com/watch?v=kOwaqR6St2I&t=...
955,senin gerçekten,975,977,senin gerçekten özel mistik gizemli bir insan ...,oKfaa43c0hA,https://www.youtube.com/watch?v=oKfaa43c0hA&t=...


In [64]:
df_word_group_time_loc_twogram_result.to_excel(f"{lang_folder.capitalize()}_Twogram_With_{word_end}_Word_{sample_num}_Youtube_{time_shift}s_Timeshift_For_Talk_Time_Result.xlsx", index=False) 

In [ ]:
df_threegram_search_result = word_group_youtube(df_youtube_sentence, df_threegram_selected_for_link["threegram"], "sentence", sample_num)
df_threegram_search_result

,search_string,start_time,end_time,sentence,video_id
0,bir şey yok,2110.706,2113.542,ya sınav yok bir şey yok bir de hava çok soğuk...,8GCGRdwg80Y
1,bir şey yok,275.258,278.422,tarık biz diye bir şey yok başlamadan bitti,HP5liOxel5M
2,bir şey yok,1265.110,1271.500,balık kırmızı balık kaçtım balık yapacak bir ş...,rXq5yNXIAac
3,bir şey yok,5176.715,5177.715,bir şey yok da,zj5ynCM8vKI
4,bir şey yok,1089.190,1098.340,lambası vardır neyse yapacak bir şey yok böyle...,50wYL3oH1NA
...,...,...,...,...,...
235,haydi bakalım şimdi,5730.773,5733.911,haydi bakalım şimdi şurada birer de acı kahve ...,jCmTFXiThTk
236,haydi bakalım şimdi,313.813,319.532,sonra haydi bakalım şimdi yeni bir afet sonras...,XDFV5Bqu6aA
237,haydi bakalım şimdi,3477.359,3479.977,haydi bakalım şimdi herkes evine,pdtUajPsAa0
238,haydi bakalım şimdi,590.400,599.220,haydi bakalım şimdi biz bunu çünkü ayırdık ama...,RISpYv3UB3k


In [ ]:
df_word_group_time_loc_threegram_result = word_group_time_loc(df_threegram_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_threegram_result

,search_string,start_time,end_time,sentence,video_id
0,bir şey yok,2111.246190,2111.831397,ya sınav yok bir şey yok bir de hava çok soğuk...,8GCGRdwg80Y
1,bir şey yok,276.288140,277.244698,tarık biz diye bir şey yok başlamadan bitti,HP5liOxel5M
2,bir şey yok,1270.025385,1271.500000,balık kırmızı balık kaçtım balık yapacak bir ş...,rXq5yNXIAac
3,bir şey yok,5176.715000,5177.572143,bir şey yok da,zj5ynCM8vKI
4,bir şey yok,1091.831237,1093.057526,lambası vardır neyse yapacak bir şey yok böyle...,50wYL3oH1NA
...,...,...,...,...,...
235,haydi bakalım şimdi,5730.773000,5731.979923,haydi bakalım şimdi şurada birer de acı kahve ...,jCmTFXiThTk
236,haydi bakalım şimdi,314.189250,315.769500,sonra haydi bakalım şimdi yeni bir afet sonras...,XDFV5Bqu6aA
237,haydi bakalım şimdi,3477.359000,3478.995250,haydi bakalım şimdi herkes evine,pdtUajPsAa0
238,haydi bakalım şimdi,590.400000,592.237500,haydi bakalım şimdi biz bunu çünkü ayırdık ama...,RISpYv3UB3k


In [ ]:
df_word_group_time_loc_threegram_result.start_time = df_word_group_time_loc_threegram_result.start_time.apply(lambda x: (x-time_shift))
df_word_group_time_loc_threegram_result.end_time = df_word_group_time_loc_threegram_result.end_time.apply(lambda x: (x+time_shift))
df_word_group_time_loc_threegram_result

,search_string,start_time,end_time,sentence,video_id
0,bir şey yok,2110.646190,2112.431397,ya sınav yok bir şey yok bir de hava çok soğuk...,8GCGRdwg80Y
1,bir şey yok,275.688140,277.844698,tarık biz diye bir şey yok başlamadan bitti,HP5liOxel5M
2,bir şey yok,1269.425385,1272.100000,balık kırmızı balık kaçtım balık yapacak bir ş...,rXq5yNXIAac
3,bir şey yok,5176.115000,5178.172143,bir şey yok da,zj5ynCM8vKI
4,bir şey yok,1091.231237,1093.657526,lambası vardır neyse yapacak bir şey yok böyle...,50wYL3oH1NA
...,...,...,...,...,...
235,haydi bakalım şimdi,5730.173000,5732.579923,haydi bakalım şimdi şurada birer de acı kahve ...,jCmTFXiThTk
236,haydi bakalım şimdi,313.589250,316.369500,sonra haydi bakalım şimdi yeni bir afet sonras...,XDFV5Bqu6aA
237,haydi bakalım şimdi,3476.759000,3479.595250,haydi bakalım şimdi herkes evine,pdtUajPsAa0
238,haydi bakalım şimdi,589.800000,592.837500,haydi bakalım şimdi biz bunu çünkü ayırdık ama...,RISpYv3UB3k


In [ ]:
df_word_group_time_loc_threegram_result.start_time = df_word_group_time_loc_threegram_result.start_time.apply(lambda x: round(x))
df_word_group_time_loc_threegram_result.end_time = df_word_group_time_loc_threegram_result.end_time.apply(lambda x: round(x))
df_word_group_time_loc_threegram_result 

,search_string,start_time,end_time,sentence,video_id
0,bir şey yok,2111,2112,ya sınav yok bir şey yok bir de hava çok soğuk...,8GCGRdwg80Y
1,bir şey yok,276,278,tarık biz diye bir şey yok başlamadan bitti,HP5liOxel5M
2,bir şey yok,1269,1272,balık kırmızı balık kaçtım balık yapacak bir ş...,rXq5yNXIAac
3,bir şey yok,5176,5178,bir şey yok da,zj5ynCM8vKI
4,bir şey yok,1091,1094,lambası vardır neyse yapacak bir şey yok böyle...,50wYL3oH1NA
...,...,...,...,...,...
235,haydi bakalım şimdi,5730,5733,haydi bakalım şimdi şurada birer de acı kahve ...,jCmTFXiThTk
236,haydi bakalım şimdi,314,316,sonra haydi bakalım şimdi yeni bir afet sonras...,XDFV5Bqu6aA
237,haydi bakalım şimdi,3477,3480,haydi bakalım şimdi herkes evine,pdtUajPsAa0
238,haydi bakalım şimdi,590,593,haydi bakalım şimdi biz bunu çünkü ayırdık ama...,RISpYv3UB3k


In [ ]:
df_word_group_time_loc_threegram_result["video_url"] = "https://www.youtube.com/watch?v="+df_word_group_time_loc_threegram_result['video_id'].map(str)+"&t="+df_word_group_time_loc_threegram_result['start_time'].map(str)+"s"
df_word_group_time_loc_threegram_result

,search_string,start_time,end_time,sentence,video_id,video_url
0,bir şey yok,2111,2112,ya sınav yok bir şey yok bir de hava çok soğuk...,8GCGRdwg80Y,https://www.youtube.com/watch?v=8GCGRdwg80Y&t=...
1,bir şey yok,276,278,tarık biz diye bir şey yok başlamadan bitti,HP5liOxel5M,https://www.youtube.com/watch?v=HP5liOxel5M&t=...
2,bir şey yok,1269,1272,balık kırmızı balık kaçtım balık yapacak bir ş...,rXq5yNXIAac,https://www.youtube.com/watch?v=rXq5yNXIAac&t=...
3,bir şey yok,5176,5178,bir şey yok da,zj5ynCM8vKI,https://www.youtube.com/watch?v=zj5ynCM8vKI&t=...
4,bir şey yok,1091,1094,lambası vardır neyse yapacak bir şey yok böyle...,50wYL3oH1NA,https://www.youtube.com/watch?v=50wYL3oH1NA&t=...
...,...,...,...,...,...,...
235,haydi bakalım şimdi,5730,5733,haydi bakalım şimdi şurada birer de acı kahve ...,jCmTFXiThTk,https://www.youtube.com/watch?v=jCmTFXiThTk&t=...
236,haydi bakalım şimdi,314,316,sonra haydi bakalım şimdi yeni bir afet sonras...,XDFV5Bqu6aA,https://www.youtube.com/watch?v=XDFV5Bqu6aA&t=...
237,haydi bakalım şimdi,3477,3480,haydi bakalım şimdi herkes evine,pdtUajPsAa0,https://www.youtube.com/watch?v=pdtUajPsAa0&t=...
238,haydi bakalım şimdi,590,593,haydi bakalım şimdi biz bunu çünkü ayırdık ama...,RISpYv3UB3k,https://www.youtube.com/watch?v=RISpYv3UB3k&t=...


In [65]:
df_word_group_time_loc_threegram_result.to_excel(f"{lang_folder.capitalize()}_Threegram_With_{word_end}_Word_{sample_num}_Youtube_{time_shift}s_Timeshift_For_Talk_Time_Result.xlsx", index=False) 

In [ ]:
df_word_search_result = word_group_youtube(df_youtube_sentence, word_list, "sentence", sample_num)
df_word_search_result

,search_string,start_time,end_time,sentence,video_id
0,bir,1901.591,1905.515,hem bu senin için daha iyi bunu anla bir kere ...,OeHxe75lMNE
1,bir,400.910,409.430,adını şöyle bir minik minik çizdim arkadaşlar ...,axWutzbx3E0
2,bir,346.581,355.497,evet bu şekilde yaklaşık bir kuzu bağırsağını ...,LiWnIynf_wg
3,bir,1631.834,1633.015,şuradan bir biber koy bana,U_tDGkeoFXs
4,bir,119.380,121.760,bir an önce bir işe girmem gerekiyordu,EvNeP7sBeus
...,...,...,...,...,...
1195,ister,5461.820,5464.780,silahını uzatıp beni ister öldür,JGuGk584FW8
1196,ister,1675.816,1677.594,bir sarılmak ister misin onlara,i_g1f4QurKs
1197,ister,456.089,457.348,bunu ister misin saniye,Qmn1S9yjxB4
1198,ister,462.613,465.796,yağı döktükten sonra ister fırça yardımı ile i...,DAZikQW876A


In [ ]:
df_word_group_time_loc_word_result = word_group_time_loc(df_word_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_word_result

,search_string,start_time,end_time,sentence,video_id
0,bir,1903.833286,1904.144714,hem bu senin için daha iyi bunu anla bir kere ...,OeHxe75lMNE
1,bir,401.951333,402.424667,adını şöyle bir minik minik çizdim arkadaşlar ...,axWutzbx3E0
2,bir,349.924500,350.621062,evet bu şekilde yaklaşık bir kuzu bağırsağını ...,LiWnIynf_wg
3,bir,1632.151962,1632.379077,şuradan bir biber koy bana,U_tDGkeoFXs
4,bir,119.380000,119.630526,bir an önce bir işe girmem gerekiyordu,EvNeP7sBeus
...,...,...,...,...,...
1195,ister,5463.670000,5464.317500,silahını uzatıp beni ister öldür,JGuGk584FW8
1196,ister,1676.504258,1676.905742,bir sarılmak ister misin onlara,i_g1f4QurKs
1197,ister,456.307957,456.691130,bunu ister misin saniye,Qmn1S9yjxB4
1198,ister,463.691983,464.069627,yağı döktükten sonra ister fırça yardımı ile i...,DAZikQW876A


In [ ]:
df_word_group_time_loc_word_result.start_time = df_word_group_time_loc_word_result.start_time.apply(lambda x: (x-time_shift))
df_word_group_time_loc_word_result.end_time = df_word_group_time_loc_word_result.end_time.apply(lambda x: (x+time_shift))
df_word_group_time_loc_word_result

,search_string,start_time,end_time,sentence,video_id
0,bir,1903.233286,1904.744714,hem bu senin için daha iyi bunu anla bir kere ...,OeHxe75lMNE
1,bir,401.351333,403.024667,adını şöyle bir minik minik çizdim arkadaşlar ...,axWutzbx3E0
2,bir,349.324500,351.221063,evet bu şekilde yaklaşık bir kuzu bağırsağını ...,LiWnIynf_wg
3,bir,1631.551962,1632.979077,şuradan bir biber koy bana,U_tDGkeoFXs
4,bir,118.780000,120.230526,bir an önce bir işe girmem gerekiyordu,EvNeP7sBeus
...,...,...,...,...,...
1195,ister,5463.070000,5464.917500,silahını uzatıp beni ister öldür,JGuGk584FW8
1196,ister,1675.904258,1677.505742,bir sarılmak ister misin onlara,i_g1f4QurKs
1197,ister,455.707957,457.291130,bunu ister misin saniye,Qmn1S9yjxB4
1198,ister,463.091983,464.669627,yağı döktükten sonra ister fırça yardımı ile i...,DAZikQW876A


In [ ]:
df_word_group_time_loc_word_result.start_time = df_word_group_time_loc_word_result.start_time.apply(lambda x: round(x))
df_word_group_time_loc_word_result.end_time = df_word_group_time_loc_word_result.end_time.apply(lambda x: round(x))
df_word_group_time_loc_word_result 

,search_string,start_time,end_time,sentence,video_id
0,bir,1903,1905,hem bu senin için daha iyi bunu anla bir kere ...,OeHxe75lMNE
1,bir,401,403,adını şöyle bir minik minik çizdim arkadaşlar ...,axWutzbx3E0
2,bir,349,351,evet bu şekilde yaklaşık bir kuzu bağırsağını ...,LiWnIynf_wg
3,bir,1632,1633,şuradan bir biber koy bana,U_tDGkeoFXs
4,bir,119,120,bir an önce bir işe girmem gerekiyordu,EvNeP7sBeus
...,...,...,...,...,...
1195,ister,5463,5465,silahını uzatıp beni ister öldür,JGuGk584FW8
1196,ister,1676,1678,bir sarılmak ister misin onlara,i_g1f4QurKs
1197,ister,456,457,bunu ister misin saniye,Qmn1S9yjxB4
1198,ister,463,465,yağı döktükten sonra ister fırça yardımı ile i...,DAZikQW876A


In [ ]:
df_word_group_time_loc_word_result["video_url"] = "https://www.youtube.com/watch?v="+df_word_group_time_loc_word_result['video_id'].map(str)+"&t="+df_word_group_time_loc_word_result['start_time'].map(str)+"s"
df_word_group_time_loc_word_result

,search_string,start_time,end_time,sentence,video_id,video_url
0,bir,1903,1905,hem bu senin için daha iyi bunu anla bir kere ...,OeHxe75lMNE,https://www.youtube.com/watch?v=OeHxe75lMNE&t=...
1,bir,401,403,adını şöyle bir minik minik çizdim arkadaşlar ...,axWutzbx3E0,https://www.youtube.com/watch?v=axWutzbx3E0&t=...
2,bir,349,351,evet bu şekilde yaklaşık bir kuzu bağırsağını ...,LiWnIynf_wg,https://www.youtube.com/watch?v=LiWnIynf_wg&t=...
3,bir,1632,1633,şuradan bir biber koy bana,U_tDGkeoFXs,https://www.youtube.com/watch?v=U_tDGkeoFXs&t=...
4,bir,119,120,bir an önce bir işe girmem gerekiyordu,EvNeP7sBeus,https://www.youtube.com/watch?v=EvNeP7sBeus&t=...
...,...,...,...,...,...,...
1195,ister,5463,5465,silahını uzatıp beni ister öldür,JGuGk584FW8,https://www.youtube.com/watch?v=JGuGk584FW8&t=...
1196,ister,1676,1678,bir sarılmak ister misin onlara,i_g1f4QurKs,https://www.youtube.com/watch?v=i_g1f4QurKs&t=...
1197,ister,456,457,bunu ister misin saniye,Qmn1S9yjxB4,https://www.youtube.com/watch?v=Qmn1S9yjxB4&t=...
1198,ister,463,465,yağı döktükten sonra ister fırça yardımı ile i...,DAZikQW876A,https://www.youtube.com/watch?v=DAZikQW876A&t=...


In [ ]:
df_word_group_time_loc_word_result.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_{sample_num}_Youtube_{time_shift}s_Timeshift_For_Talk_Time_Result.xlsx", index=False)

#### Copy Move And Delete

In [74]:
output_file = glob.glob(f"{lang_folder.capitalize()}_*_{sample_num}_Youtube_{time_shift}s_Timeshift_For_Talk_Time_Result.xlsx")
output_file

['Turkish_Twogram_With_200_Word_6_Youtube_0.6s_Timeshift_For_Talk_Time_Result.xlsx',
 'Turkish_Threegram_With_200_Word_6_Youtube_0.6s_Timeshift_For_Talk_Time_Result.xlsx',
 'Turkish_200_Word_6_Youtube_0.6s_Timeshift_For_Talk_Time_Result.xlsx']

In [75]:
for y in output_file:
    source = y # source directory
    destination = path
    shutil.copy2(source, destination)

In [76]:
for z in output_file:
    try:
        os.remove(z)
    except:
        pass